In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Module 5: Bias-Variance and Cross-Validation
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [1]:
import pandas as pd
import os

week = 4  # from last module

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table = pd.read_csv(home_path + file_path + file_name)

In [2]:
os.chdir(home_path + file_path + 'datascience_1/')
!git pull

Already up-to-date.


In [3]:
#load the lbirary from content this week

import sys
sys.path.append(home_path + file_path + 'datascience_1/') 
from week5 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 compute_prediction	 compute_training	 f1	 find_best_splitter	 generate_table	 
gig	 gini	 informedness	 k_fold	 predictor_case	 probabilities	 tree_predictor	 


In [6]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+,tree_1,tree_1_type,tree_2,tree_2_type,tree_3,tree_3_type,tree_4,tree_4_type,tree_5,tree_5_type
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,Low,1,0,0,0,0,1,0,0,1,false_positive,1,false_positive,1,false_positive,1,false_positive,1,false_positive
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0,1,true_positive,1,true_positive,1,true_positive,1,true_positive,1,true_positive


In [7]:
loan_table.columns.values

array(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Loan_Status', 'no_lam', 'filled_lam', 'pa_Rural', 'pa_Semiurban',
       'pa_Urban', 'pa_nan', 'bin_lam', 'bin_Low', 'bin_Average',
       'bin_High', 'ch_bad', 'ch_good', 'ch_nan', 'apin_binned',
       'apin_Low', 'apin_Average', 'apin_High', 'apin_nan', 'dep_0',
       'dep_1', 'dep_2', 'dep_3+', 'tree_1', 'tree_1_type', 'tree_2',
       'tree_2_type', 'tree_3', 'tree_3_type', 'tree_4', 'tree_4_type',
       'tree_5', 'tree_5_type'], dtype=object)

<hr>
<h1>
1: Wrangle
</h1>
<p>
<div class=h1_cell>
<p>
Drop these columns: 'tree_1', 'tree_1_type', etc.
<p>
</div>

In [8]:
loan_table = loan_table.drop(['tree_1', 'tree_1_type', 
                              'tree_2', 'tree_2_type',
                              'tree_3', 'tree_3_type',
                              'tree_4', 'tree_4_type',
                              'tree_5', 'tree_5_type'], axis=1)

In [9]:
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,bin_lam,bin_Low,bin_Average,bin_High,ch_bad,ch_good,ch_nan,apin_binned,apin_Low,apin_Average,apin_High,apin_nan,dep_0,dep_1,dep_2,dep_3+
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,Low,1,0,0,0,1,0,0,0


<hr>
<h1>
2: Explore depth parameter with 10-folding
</h1>
<p>
<div class=h1_cell>
<p>
In the last module you were asked to explore depths from 1 to 5 without any cross-validation. Go ahead and do it again but using K-folding with K=10.
<p>
First, define the columns to use. I do that for you below.
</div>

In [10]:
column_candidates = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'apin_Low', 'apin_High', 'apin_Average',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'bin_Low', 'bin_Average', 'bin_High',
        #Credit_History
        'ch_bad', 'ch_good']

<hr>
<h2>
Show your results below
</h2>
<p>
<div class=h1_cell>
<p>
Go ahead and get the means of accuracy, f1 and informedness on depths 1-5 with 10-folding.
</div>

In [11]:
table1 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth':1}, column_candidates)
table2 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth':2}, column_candidates)
table3 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth':3}, column_candidates)
table4 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth':4}, column_candidates)
table5 = k_fold(loan_table, 10, 'Loan_Status', {'max-depth':5}, column_candidates)

print(table1.head())


/home/lain/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


          name  true_positive  false_positive  true_negative  false_negative  \
0  fold 1 test           39.0            14.0            8.0             NaN   
1  fold 2 test           43.0             9.0            9.0             NaN   
2  fold 3 test           38.0            13.0            8.0             2.0   
3  fold 4 test           43.0            13.0            4.0             1.0   
4  fold 5 test           42.0            11.0            7.0             1.0   

   accuracy        f1  informedness  
0  0.770492  0.847826      0.363636  
1  0.852459  0.905263      0.500000  
2  0.754098  0.835165      0.330952  
3  0.770492  0.860000      0.212567  
4  0.803279  0.875000      0.365633  


<hr>
<h1>
Check your results
</h1>
<p>
<div class=h1_cell>
<p>
Check your scores against mine in the cell below. The first row is from last module using no cross-validation. The second row (i.e., 10:) is what I got when using 10-folding.
<p>
Every one of my scores with 10-folding went down slightly.
</div>

<hr>
<h1>
3: Random slices
</h1>
<p>
<div class=h1_cell>
<p>
I'd like you to try one more thing. The `k_fold` function takes slices sequentially. I'd like you to write a function `k_fold_random` that takes random slices *without replacement*. Without replacement means that each slice should have a unique set of rows, i.e., no slices share a row
<p>
Each slice should be roughly 1/k percent of the entire table.
<p>
To solve this problem, I found it easiest to play around with loops in a notebook cell before trying to write the full function. My target was the equivalent of this but now with random slices:
<pre>
<code>
    total_len = len(table.index)
    slice_size = int(total_len/(1.0`*`k))
    slices = []

    #generate the slices
    for i in range(k-1):
        a_slice =  table[i`*`slice_size:(i+1)`*`slice_size]
        slices.append( a_slice )
    slices.append( table[(k-1)`*`slice_size:] )
</code>
</pre>
<p>
I copied this code into a cell and then started to play around with various ideas until I got one I liked.
<p>
To make sure you are generating slices without replacement, I'll give you a function you can call on your slices list.
</div>

In [12]:
#Determine if slices are mutually exclusive
def verify_unique(slices):
    for i, a_slice in enumerate(slices[:-1]):
        a_set = set(a_slice.index)
        for j, b_slice in enumerate(slices[i+1:]):
            b_set = set(b_slice.index)
            int_set = a_set.intersection(b_set)  # should be empty set as result
            print((i,j,int_set))
    return None

In [18]:
# here is sequential slice code from k_fold if you want to use it as base.
# modify it to produce slices with random rows in each slice.

import numpy as np ## need this for numpy to work, I assume it's allowed since pandas is also allowed

from random import shuffle 

#table = loan_table
table = loan_table.reindex(np.random.permutation(loan_table.index)) ## from https://stackoverflow.com/questions/15772009/shuffling-permutating-a-dataframe-in-pandas/15772330
k = 5

total_len = len(table.index)
split_size = int(total_len/(1.0*k))
slices = []


#generate the slices
for i in range(k-1): #for 1-5... 
    a_slice =  table[i*split_size:(i+1)*split_size] #0 * 114
    slices.append( a_slice )
slices.append( table[(k-1)*split_size:] ) # append the final slice

#print(slices)




verify_unique(slices)  # should see empty sets all the way down

(0, 0, set([]))
(0, 1, set([]))
(0, 2, set([]))
(0, 3, set([]))
(1, 0, set([]))
(1, 1, set([]))
(1, 2, set([]))
(2, 0, set([]))
(2, 1, set([]))
(3, 0, set([]))


<h2>
Rest of k_fold code should work
</h2>
<p>
<div class=h1_cell>
<p>
Once you are generating slices randomly, you will have k test slices. You can go ahead and place your code into `k_fold_random`. You should now be good to go.
<p>
One problem with doing random selections is that we will get different results every time we run our code. We will see a way to get repeatable results in next module.
</div>

In [14]:
def k_fold_random(table, k, target, hypers, candidate_columns):
    result_columns = ['name', 'true_positive', 'false_positive', 'true_negative', 'false_negative', 'accuracy', 'f1', 'informedness']
    k_fold_results_table = pd.DataFrame(columns=result_columns)
    
    table = table.reindex(np.random.permutation(loan_table.index))
    
    total_len = len(table.index)
    slice_size = int(total_len/(1.0*k))
    slices = []

    #generate the slices
    for i in range(k-1):
        a_slice =  table[i*slice_size:(i+1)*slice_size]
        slices.append( a_slice )
    slices.append( table[(k-1)*slice_size:] )  # whatever is left
    
    #generate test results
    for i in range(k):
        test_table = slices[i]
        train_table = compute_training(slices, i)
        fold_tree = build_tree_iter(train_table, candidate_columns, target, hypers)  # train
        fold_cases = caser(test_table, fold_tree, target)  # test

        k_fold_results_table = k_fold_results_table.append(fold_cases,ignore_index=True)
        end = k_fold_results_table.last_valid_index()
        k_fold_results_table.name.iloc[end] =  'fold '+str(i+1)+' test'
        k_fold_results_table.accuracy.iloc[end] =  accuracy(fold_cases)
        k_fold_results_table.f1.iloc[end] =  f1(fold_cases)
        k_fold_results_table.informedness.iloc[end] =  informedness(fold_cases)
        
    k_fold_results_table.__doc__ = str(hypers)  # adds comment to remind me of hyper params used
    return k_fold_results_table

<h2>
Test on depths 1-5
</h2>
<p>
<div class=h1_cell>
<p>
You tried `k_fold` on depths 1-5. Now try `k_fold_random` and record your means. You can compare against what I got.
</div>

In [22]:
#table1 = k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth':1}, column_candidates)
#table2 = k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth':2}, column_candidates)
#table3 = k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth':3}, column_candidates)
#table4 = k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth':4}, column_candidates)
table5 = k_fold_random(loan_table, 10, 'Loan_Status', {'max-depth':5}, column_candidates)

table5.head()

,name,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
0,fold 1 test,39.0,10.0,10.0,2.0,0.803279,0.866667,0.451220
1,fold 2 test,45.0,11.0,4.0,1.0,0.803279,0.882353,0.244928
2,fold 3 test,40.0,10.0,10.0,1.0,0.819672,0.879121,0.475610
3,fold 4 test,39.0,9.0,11.0,2.0,0.819672,0.876404,0.501220
4,fold 5 test,40.0,11.0,8.0,2.0,0.786885,0.860215,0.373434


<div class=h1_cell>

Here are my results. Your's will likely vary given nature of random selection.
</div>

<hr>
<h1>Write it out</h1>
<div class=h1_cell>
<p>
Save the table so can use it in next module.
</div>

In [17]:
import os

week = 5 # change this each week

home_path =  os.path.expanduser('~')

file_path = '/Desktop/CIS399/'

file_name = 'loan_wrangled_w'+str(week)+'.csv'

loan_table.to_csv(home_path + file_path + file_name, index=False)